In [4]:
import imports
import importlib
importlib.reload(imports)
from imports import *

# torch.cuda.is_available(), torch.rand(1).to('cuda:0')

In [4]:
 params['INFECT_TYPE_TO_ID']

{'nothing': 0,
 'REPLACE_UNDEF': 1,
 'INCORRECT_PUNCT_RANDOM_PLACE': 2,
 'INCORRECT_PUNCT_CENTER': 3,
 'CORRECT_PUNCT_CENTER': 4,
 'CORRECT_PUNCT_RIGHT': 5}

In [10]:
from expriments_for_vkr import *
from repr import Model
import reprlib
setattr(reprlib, "Model", Model)

print('cut;model;f-score;f-score_space;f-score_comma;f-score_dot;'
     'precision_space;precision_comma;precision_dot;'
     'recall_space;recall_comma;recall_dot')

def f(x):
    return "{:.3f}".format(x) # .replace(".", ",")

def ff(x):
    return ";".join(map(f, x))

def run_valid(ccut, prefix):
        with open(f"{prefix}/some_model_CLASS.dill", "rb") as file:
            Model = dill.load(file)
        setattr(reprlib, "Model", Model)

        # model = torch.load(f"{prefix}/some_model.pt", map_location=torch.device('cpu')) #).v1", map_location=torch.device('cuda:0'))
        model = torch.load(f"{prefix}/some_model.pt", map_location=torch.device('cuda:0'))
        # print(model)
        with open(f"{prefix}/storage_path.dill", "rb") as file:
            storage_path = dill.load(file)
        with open(f"{prefix}/is_infected_test.dill", "rb") as file:
            is_infected = dill.load(file)

        storage = Storage(str(storage_path), enable_watcher=False)
        params = storage.get_meta("params")
        is_infected = storage.get_meta("is_infected_test")
            
        if ccut == 'all':
            cut = is_infected != -1
        else:
            cut = is_infected == params['INFECT_TYPE_TO_ID'][ccut]
        x_test = storage.get_meta("x_test").float()[cut]
        y_test = storage.get_meta("y_test").float()[cut]
        text_res = storage.get_meta("text_res_test")

        with torch.no_grad():
            model.eval()
            model.cuda()
            x_test_cuda = x_test.cuda()
            
            y_test_pred = []
            for x in torch.split(x_test_cuda, x_test_cuda.shape[0] // 10):
                y_test_pred.append(model(x))
            y_test_pred = torch.cat(y_test_pred)
            
            _, y_pred_tags = torch.max(y_test_pred, dim = 1)
            y_pred_ = y_pred_tags.cpu().numpy()
            _, y_test_tags = torch.max(y_test, dim = 1)
            y_test_ = y_test_tags.cpu().numpy()
        del model

        import pandas as pd
        import seaborn as sns
        from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score


        print(ccut + ";" + prefix, end=';')
        print(f(f1_score(y_test_, y_pred_, average='micro')), end=';')
        print(ff(f1_score(y_test_, y_pred_, average=None)), end=';')
        print(ff(precision_score(y_test_, y_pred_, average=None)), end=';')
        print(ff(recall_score(y_test_, y_pred_, average=None)), end='')
        print()
        
        del x_test_cuda
        del y_test_pred
        del y_pred_tags
        import gc
        gc.collect()
        
# for ccut in ['all', 'nothing']:
#     for prefix in ["results_final1_linear_model", "results_final2_lstm_model",
#                    "results_final3_encoder_lstm_model", 
#                    "results_big_model_GOOD_2022-05-15",
#                     "results_final4_tranformer_model",
#                    "results_no_morph_final1_linear_model", 
#                    "results_no_morph_final2_lstm_model",
#                    "results_no_motph_final3_transformer_lstm_model",
#                    "results_no_morph_final4_transformer_model"]:
#         run_valid(ccut, prefix)

# for prefix in ['results_final3_encoder_lstm_model', 'results_final4_tranformer_model']:
#     for ccut in ['all', 'nothing', 'REPLACE_UNDEF', 
#                   'INCORRECT_PUNCT_RANDOM_PLACE', 
#                   'INCORRECT_PUNCT_CENTER', 
#                   'CORRECT_PUNCT_CENTER',
#                     'CORRECT_PUNCT_RIGHT']:
#         run_valid(ccut, prefix)

cut;model;f-score;f-score_space;f-score_comma;f-score_dot;precision_space;precision_comma;precision_dot;recall_space;recall_comma;recall_dot


In [11]:
run_valid('nothing', 'results_big_model_GOOD_2022-05-15')

nothing;results_big_model_GOOD_2022-05-15;0.960;0.972;0.917;0.968;0.964;0.935;0.982;0.980;0.900;0.954


In [8]:
with open(f"results_big_model_GOOD_2022-05-15/some_model_CLASS.dill", "rb") as file:
    Model = dill.load(file)
import reprlib
setattr(reprlib, "Model", Model)

# model = torch.load(f"{prefix}/some_model.pt", map_location=torch.device('cpu')) #).v1", map_location=torch.device('cuda:0'))
model = torch.load(f"results_big_model_GOOD_2022-05-15/some_model.pt", map_location=torch.device('cpu'))#, map_location=torch.device('cuda:0'))

Either FairScale or torch distributed is not available, MixtureOfExperts will not be exposed. Please install them if you would like to use MoE


In [9]:
from utils import count_parameters
count_parameters(model)

37.13557052612305

In [1]:
import torch

In [2]:
import torch

model, example_texts, languages, punct, apply_te = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                                                  model='silero_te')

input_text = input('Enter input text\n')
apply_te(input_text, lan='en')

Using cache found in /home/misha-sh/.cache/torch/hub/snakers4_silero-models_master


Enter input text
 1


'1'

In [3]:
count_parameters(model.model)

0.5322074890136719

In [18]:

def calculate_diff2(text, text_res):
  res = defaultdict(lambda: 0)

  def is_punctuation(c):
      return c in ".,-!?—:\"'()&_«»*:;…{#}––“„"

  def sink_add(c):
    nonlocal res
    if c in '.,':
        res['added ' + c] += 1

  def sink_remove(c):
    nonlocal res
    if c in '.,':
        res['removed ' + c] += 1

  def sink_change(c1, c2):
    nonlocal res
    if c1 in '.,' and c2 in '.,':
        res['changed ' + c1 + " with " + c2] += 1

  i = 0
  j = 0
  while True:
      if i >= len(text): break
      if j >= len(text_res): break
      if text[i] == ' ':
            i += 1
            continue
      if text_res[j] == ' ':
            j += 1
            continue
      # print(text[i], text_res[j])
      if text[i] == text_res[j]:
          if is_punctuation(text[i]) and text[i] in '.,':
             res['not changed ' + text[i]] += 1
          i += 1
          j += 1
          continue

      if is_punctuation(text[i]) and is_punctuation(text_res[j]):
        sink_change(text[i], text_res[j])
        i += 1
        j += 1
        continue

      if is_punctuation(text[i]):
        sink_remove(text[i])
        i += 1
        continue

      if is_punctuation(text_res[j]):
        sink_add(text_res[j])
        j += 1
        continue

      raise Exception("Change not in punctuation", text[i], text_res[j], "at ", i, j)

  while i < len(text):
    if text[i] == ' ':
        i += 1
        continue
    # print("remaining: ", text[i])
    assert is_punctuation(text[i])
    sink_remove(text[i])
    i += 1

  while j < len(text_res):
    if text_res[j] == ' ':
        j += 1
        continue
    # print("remaining(2): ",text_res[j])
    assert is_punctuation(text_res[j])
    sink_add(text_res[j])
    j += 1

  res['possible punctuation places'] = len(list(tokenize(text)))

  return res

In [139]:
calculate_diff2(text.lower(), res.lower())

AttributeError: 'collections.defaultdict' object has no attribute 'lower'

In [131]:
# text = text_to_infer[500:2000]
# txt = text.replace(".", " ")\
#         .replace(",", " ").replace("-", " ").replace("!", " ")\
#         .replace("?", " ").replace("—", " ")
# txt

In [132]:
# res = "\n".join(map(apply_te, txt.lower().split("\n")))
# res

In [124]:
txt[20:]

'еяния изображенные в Ветхом завете говорят о его злопамятности несправедливости '

In [123]:
res[20:]

' изображенные в ветхом завете, говорят о его злопамятности, несправедливости.'

In [78]:
"\n".join(map(apply_te, txt.lower().split("\n")))

'Знакомство все его деяния, изображенные в Ветхом Завете говорят о его злопамятности, несправедливости, мелочности, безжалостности, мстительности. Он только и делает, что караеткарает за ничтожные проступки с тысячекратной строгостью; карает невинных младенцев за проступки их родителей; карает ни в чем не п'

In [33]:
text_to_infer

'марк твен \nплан города парижа\n\n\n\nмарк твен\nплан города парижа\n\n к читателю  \nприлагаемый план говорит сам за себя  идея этого плана не оригинальна  она заимствована мною из столичных газет \n\n\n\nплан парижа \nэто произведение искусства  если только можно назвать его так  не претендует на какиелибо иные достоинства  кроме точности  как правило  недостатки подобных планов объясняются тем  что их авторы  повидимому  уделяют больше внимания художественности исполнения  нежели географической достоверности \nпоскольку это моя первая попытка составить и выгравировать план    да и вообще подвизаться на поприще искусства    то мне очень польстили похвалы и восхищение  вызванные этой работой  и особенно трогательно  что самые восторженные отзывы исходят от людей  ничего не смыслящих в искусстве \nпо незначительному недосмотру я выгравировал план так  что всем  кроме левши  придется читать его шиворотнавыворот  я забыл  что для того  чтобы получить правильный отпечаток  надо все и чер

In [30]:
rres

'Марк Твен!\nПлан города Парижа.\n\n\n\nМарк Твен!\nПлан города Парижа.\n\nК читателю?\nПрилагаемый план говорит сам за себя. Идея этого плана не оригинальна. Она заимствована мною из столичных газет.\n\n\n\nПлан парижа.\nЭто произведение искусства, если только можно назвать его так не претендует на какиелибо иные достоинства, кроме точности. Как правило, недостатки подобных планов объясняются тем, что их авторы повидимому уделяют больше внимания художественности исполнения, нежели географической достоверности.\nПоскольку это моя первая попытка составить и выгравировать план, да и вообще подвизаться на поприще искусства. То мне очень польстили похвалы и восхищение. Вызванные этой работой и особенно трогательно, что самые восторженные отзывы исходят от людей, ничего не смыслящих в искусстве.\nПо незначительному недосмотру я выгравировал план, так что всем, кроме Левши придется читать его шиворотнавыворот. Я забыл, что для того, чтобы получить правильный отпечаток надо все и чертить и гр

In [32]:
txt

'Марк Твен \nПлан города Парижа\n\n\n\nМарк Твен\nПлан города Парижа\n\n(К читателю) \nПрилагаемый план говорит сам за себя. Идея этого плана не оригинальна, она заимствована мною из столичных газет.\n\n\n\nПлан Парижа \nЭто произведение искусства (если только можно назвать его так) не претендует на какиелибо иные достоинства, кроме точности. Как правило, недостатки подобных планов объясняются тем, что их авторы, повидимому, уделяют больше внимания художественности исполнения, нежели географической достоверности.\nПоскольку это моя первая попытка составить и выгравировать план, – да и вообще подвизаться на поприще искусства, – то мне очень польстили похвалы и восхищение, вызванные этой работой. И особенно трогательно, что самые восторженные отзывы исходят от людей, ничего не смыслящих в искусстве.\nПо незначительному недосмотру я выгравировал план так, что всем, кроме левши, придется читать его шиворотнавыворот. Я забыл, что для того, чтобы получить правильный отпечаток, надо все и чер

In [29]:
import joblib
f = round(joblib.cpu_count())
print(f)
torch.set_num_threads(f)

import glob
from striprtf.striprtf import rtf_to_text
from tqdm.notebook import tqdm
def dicts_sum(dict1, dict2):
  for key in dict2:
    dict1[key] += dict2[key]
  return dict1

clear_punctuation = False
res = defaultdict(lambda: 0)
i = 0
for rtf_path in tqdm(glob.glob("../validation/Mark Tven/Mark Tven rtf/*.rtf")):
    with open(rtf_path, "rb") as rtf_file:
        encoded = rtf_file.read()
        try:
            rtf = encoded.decode('cp1251')
            ttxt = rtf_to_text(rtf).replace('\xa0', " ")#[:2000]
            for j in range(0, len(ttxt), 20000):
                try:
                    txt = ttxt[j:j+20000]
                    if clear_punctuation:
                       text_to_infer = txt.replace(". ", " ").replace(", ", " ")
                    else:
                       text_to_infer = txt

                    text_to_infer = text_to_infer.replace(".", " ")\
                        .replace(",", " ").replace("-", " ").replace("!", " ")\
                        .replace("?", " ").replace("—", " ").replace(":", " ")\
                        .replace("\"", " ").replace("'", " ")\
                        .replace("(", " ").replace(")", " ").replace("&", " ")\
                        .replace("_", " ").replace("«", " ").replace("»", " ")\
                        .replace("*", " ").replace(";", " ").replace(":", " ")\
                        .replace("…", " ").replace("{", " ").replace("}", " ")\
                        .replace("#", " ").replace("–", " ").replace("“", " ")\
                        .replace("„", " ").lower()
                    # print(text_to_infer)
                    rres = "\n".join([apply_te(jj, 'ru') for jj in text_to_infer.split("\n")])
                    # print("\n", txt)
                    # print("\n", rres, "\n")
                    diff = calculate_diff2(txt.lower(), rres.lower())
                    # print(diff)
                    res = dicts_sum(res, diff)
                except Exception as ex:
                    print("skipped ", j, rtf_path, len(encoded), ex)
        except Exception as ex:
            print("skipped ", rtf_path, len(encoded), ex)
            # raise
        i += 1
        # break
        # if i> 2: break

print(res)

64


  0%|          | 0/148 [00:00<?, ?it/s]

skipped  0 ../validation/Mark Tven/Mark Tven rtf/Tven_Uchenyie_skazochki_dlya_primernyih_pozhilyih_malchikov_i_devochek.149907.rtf 71164 
skipped  0 ../validation/Mark Tven/Mark Tven rtf/Tven_Moya_pervaya_beseda_s_Artimesom_Uordom.149893.rtf 33391 ('Change not in punctuation', 'я', 'в', 'at ', 2178, 2139)
skipped  20000 ../validation/Mark Tven/Mark Tven rtf/Tven_Nalegke.149894.rtf 1013017 ('Change not in punctuation', 'в', 'п', 'at ', 14622, 14388)
skipped  80000 ../validation/Mark Tven/Mark Tven rtf/Tven_Nalegke.149894.rtf 1013017 ('Change not in punctuation', 'м', 'п', 'at ', 19188, 18861)
skipped  220000 ../validation/Mark Tven/Mark Tven rtf/Tven_Nalegke.149894.rtf 1013017 ('Change not in punctuation', 'с', '\n', 'at ', 12223, 11988)
skipped  400000 ../validation/Mark Tven/Mark Tven rtf/Tven_Nalegke.149894.rtf 1013017 ('Change not in punctuation', 'и', '\n', 'at ', 14505, 14261)
skipped  520000 ../validation/Mark Tven/Mark Tven rtf/Tven_Nalegke.149894.rtf 1013017 ('Change not in pun

In [53]:
a = list(sorted([i.split('/')[-1].split('.')[0] for i in glob.glob("../validation/Mark Tven/Mark Tven rtf/*.rtf")]))
a = a[1:]
b = "\n".join([str(i + 1) + ". " + a[i]#.split('Tven_')[1]
               for i in range(len(a))])
print(b)

1. Tven_Bankovyiy_bilet_v_1000000_funtov_sterlingov
2. Tven_Basnya
3. Tven_Beglyie_zametki_ob_odnoy_prazdnoy_poezdke
4. Tven_Bibleyskie_poucheniya_i_religioznaya_taktika
5. Tven_Chelovek_kotoryiy_sovratil_Gedliberg
6. Tven_Cheloveku_Hodyaschemu_vo_Tme
7. Tven_Chernokozhiy_sluga_generala_Vashingtona
8. Tven_Detektiv_s_dvoynyim_pritselom
9. Tven_Dikovinnyiy_son
10. Tven_Dnevnik_Adama_Fragmentyi_
11. Tven_Dnevnik_Evyi
12. Tven_Dobroe_slovo_satanyi
13. Tven_Dobryiy_aptekar
14. Tven_Dva_fragmenta_iz_zapreschennoy_knigi_ozaglavlennoy_Vzglyad_na_istoriyu
15. Tven_Dve_kratkie_lektsii_po_nauke
16. Tven_Epidemiya
17. Tven_Frantsuzyi_i_komanchi
18. Tven_Hristianskaya_nauka
19. Tven_Inostrannaya_perepiska
20. Tven_Ispravlennyie_nekrologi
21. Tven_Istoriya_Meymi_Grant_devochki-missionera
22. Tven_Istoriya_s_privideniem
23. Tven_Iz_Avtobiografii
24. Tven_Iz_zapisnyih_knizhek_1865-1905
25. Tven_Kak_izbavitsya_ot_rechey
26. Tven_Kak_lechit_prostudu
27. Tven_Kak_menya_proveli_v_Nyuarke
28. Tven_Kak_men

32

64

In [196]:
punct

'.,-!?—'

In [55]:
clear_punctuation = False
res = defaultdict(lambda: 0)
i = 0
ss = 0
for rtf_path in tqdm(glob.glob("../validation/Mark Tven/Mark Tven rtf/*.rtf")):
    with open(rtf_path, "rb") as rtf_file:
        encoded = rtf_file.read()
        try:
            rtf = encoded.decode('cp1251')
            ttxt = rtf_to_text(rtf).replace('\xa0', " ")
            ss += len(ttxt)
        except: continue
ss - 40 * 10000

  0%|          | 0/148 [00:00<?, ?it/s]

12083264

In [67]:
import zipfile
with zipfile.ZipFile( "./cache2/writers.zip", 'r') as writers_dir:
    for file in writers_dir.filelist:
        f = file.filename
        good_file = ('prose' in f or 'publicism' in f) and 'txt' in f and 'Blok' not in f # and 'Tolstoy' in f
        if not good_file: continue
        print(file.filename)

prose/Bryusov/╨É╨╗╤é╨░╤Ç╤î ╨┐╨╛╨▒╨╡╨┤╤ï.txt
prose/Bryusov/╨æ╨╡╨╝╨╛╨╗╤î.txt
prose/Bryusov/╨Æ ╨▒╨░╤ê╨╜╨╡.txt
prose/Bryusov/╨Æ ╨╖╨╡╤Ç╨║╨░╨╗╨╡.txt
prose/Bryusov/╨Æ ╨┐╨╛╨┤╨╖╨╡╨╝╨╜╨╛╨╣ ╤é╤Ä╤Ç╤î╨╝╨╡.txt
prose/Bryusov/╨Æ╨╛╤ü╤ü╤é╨░╨╜╨╕╨╡ ╨╝╨░╤ê╨╕╨╜.txt
prose/Bryusov/╨ô╨╛╤Ç╨░ ╨ù╨▓╨╡╨╖╨┤╤ï.txt
prose/Bryusov/╨ö╨╡╨║╨░╨┤╨╡╨╜╤é.txt
prose/Bryusov/╨ö╨╕╤é╤Å ╨╕ ╨▒╨╡╨╖╤â╨╝╨╡╤å.txt
prose/Bryusov/╨ù╨░ ╤ü╨╡╨▒╤Å ╨╕╨╗╨╕ ╨╖╨░ ╨┤╤Ç╤â╨│╤â╤Ä.txt
prose/Bryusov/╨ù╨░╤ë╨╕╤é╨░.txt
prose/Bryusov/╨£╨╛╤å╨░╤Ç╤é.txt
prose/Bryusov/╨£╤Ç╨░╨╝╨╛╤Ç╨╜╨░╤Å ╨│╨╛╨╗╨╛╨▓╨║╨░.txt
prose/Bryusov/╨¥╨╛╤ç╨╜╨╛╨╡ ╨┐╤â╤é╨╡╤ê╨╡╤ü╤é╨▓╨╕╨╡.txt
prose/Bryusov/╨₧╨▒╤Ç╤â╤ç╨╡╨╜╨╕╨╡ ╨ö╨░╤ê╨╕.txt
prose/Bryusov/╨₧╨│╨╜╨╡╨╜╨╜╤ï╨╣ ╨░╨╜╨│╨╡╨╗.txt
prose/Bryusov/╨ƒ╨╡╤Ç╨▓╨░╤Å ╨╗╤Ä╨▒╨╛╨▓╤î.txt
prose/Bryusov/╨ƒ╨╛╨┤ ╨í╤é╨░╤Ç╤ï╨╝ ╨╝╨╛╤ü╤é╨╛╨╝.txt
prose/Bryusov/╨ƒ╨╛╤ü╨╗╨╡╨┤╨╜╨╕╨╡ ╨╝╤â╤ç╨╡╨╜╨╕╨║╨╕.txt
prose/Bryusov/╨ƒ╨╛╤ü╨╗╨╡╨┤╨╜╨╕╨╡ ╤ü╤é╤Ç╨░╨╜╨╕╤å╤ï ╨╕╨╖ ╨┤╨╜╨╡╨▓╨╜╨╕╨║╨░ ╨╢╨╡╨╜╤ë╨╕╨╜╤ï.txt
prose/Bryusov/╨ƒ╤Ç╨╡╨┤╤ü╨╝╨╡╤Ç╤é╨╜╤ï╨╣ ╨▒╤Ç╨╡╨┤ ╨Æ╨╡╤Ç╨│╨╕╨╗╨╕╤Å.txt
prose/Bryu

In [65]:
from dataset_lib import get_lenta_records, get_writers_multi_stream
from razdel import tokenize
tokens = 0
for i in get_writers_multi_stream().limit(10_000):
    tokens += len(list(tokenize(i)))
tokens

3579040

In [59]:
storage = Storage("cache2/storage2", enable_watcher=False)
storage.get("x", 0).shape

torch.Size([1675, 32, 489])

In [60]:
1675 * 2160

3618000

In [21]:
import numpy as np
import random
import pymorphy3
import numpy as np
import math
import pickle
from razdel import tokenize

from dataset_builder import calculate_word_features_for_tokens, PAD_TOKEN,get_word_features
from inference import torch_model_runner, onnx_model_runner, infer
import onnxruntime as ort
# print("PREFIX\n" * 100)
def onnx_model_runner(path):
    sess_opt = ort.SessionOptions()
    sess_opt.intra_op_num_threads = 1
    sess_opt.inter_op_num_threads = 1
    ort_sess = ort.InferenceSession(path, sess_opt)
    def func(input):
        return ort_sess.run(None, {'input': np.array(input) })[0]
    return func

# onnx_model = onnx_model_runner("results writers big/model.onnx")
prefix = "results_big_model_GOOD_2022-05-15"
onnx_model = onnx_model_runner(f"{prefix}/model.onnx")
with open("params.pickle", "rb") as f:
    params = pickle.load(f)

# print(onnx_model(dummy_input.numpy()))

class jsinfer:
    async def infer(arr):
        class wrapper:
            def to_py():
                return onnx_model(arr)
        return wrapper

from stream import Stream
import functools
from collections import deque
import random
random.seed(42)

@functools.lru_cache(maxsize=128)
def get_word_features_cached(word):
    return get_word_features(word, params).numpy()

class Substr:
    def __init__(self, text):
        self.text = text
    def __repr__(self) -> str:
        return f"Substring(-1, -1, {self.text})"

def d_as_str(d):
  return "<" + " ".join(map(lambda text: text.text, d))+ ">"

async def infer_optimal(params, text):
  # print("INFERCENC IS WIERD\n" * 10)
  res = []
  last_inserted_pos = 0
  def sink(token, log=False):
    nonlocal last_inserted_pos
    if token.text == "PAD": return
    if log: print('sink', token)
    if isinstance(token, Substr):
      res.append(token.text)
      if log: print("added1 ", f"`{token.text}`", token)
    else:
      if last_inserted_pos != token.start:
        res.append(text[last_inserted_pos: token.start])
        if log: print("added2 ", f"`{text[last_inserted_pos: token.start]}`", last_inserted_pos, token.start)
      last_inserted_pos = token.stop
      res.append(token.text)
      if log: print("added3 ", f"`{token.text}`", token)

  def skip(token, log=False):
    nonlocal last_inserted_pos
    last_inserted_pos = token.stop
    if log: print('skip', token)

  def sink_remaining():
     res.append(text[last_inserted_pos:])


  async def predict_on_tokens(window_left, window_right, return_probas):
    features = [get_word_features_cached(i.text) for i in Stream(window_left).chain(window_right)]
    features_for_batch = np.stack((features, ))
    arr = np.ascontiguousarray(features_for_batch, dtype=np.float32)
    output_probas = np.array((await jsinfer.infer(arr)).to_py())
    # output_probas[0][0] += 2.
    if return_probas:
      return params["ID_TO_PUNCTUATION"], output_probas
    punct_idx = np.argmax(output_probas).item()
    punct = params["ID_TO_PUNCTUATION"][punct_idx]
    return punct


  window_left = deque()
  window_right = deque()
  log = False
  skip_next = False
  for i in Stream.repeat(Substr(PAD_TOKEN), params['INPUT_WORDS_CNT_LEFT']) \
      .chain(Stream(tokenize(text))) \
      .chain(Stream.repeat(Substr(PAD_TOKEN), params["INPUT_WORDS_CNT_RIGHT"])):
    window_right.append(i)
    if len(window_right) <= params["INPUT_WORDS_CNT_RIGHT"]:
        continue
    assert len(window_right) == params["INPUT_WORDS_CNT_RIGHT"] + 1

    next_ = window_right.popleft()
    sink(next_)
    window_left.append(next_)
    if len(window_left) < params['INPUT_WORDS_CNT_LEFT']:
      continue

    assert len(window_left) == params["INPUT_WORDS_CNT_LEFT"]
    assert len(window_right) == params["INPUT_WORDS_CNT_RIGHT"]

    if skip_next:
      prediction = "$skip"
    else:
      # params["ID_TO_PUNCTUATION"], output_probas
      prediction = await predict_on_tokens(window_left, window_right, return_probas=False)


    #random.choice([" ", "."])
    if log: print(d_as_str(window_left).rjust(100), prediction.center(6), d_as_str(window_right))

    def is_replaceable_punct(punct):
      return punct in ',.'

    if prediction == "$skip":
      pass
    elif prediction != "$empty":
      if is_replaceable_punct(window_right[0].text):
        if window_right[0].text != prediction:
          window_right[0].text = prediction
      else:
        window_left.append(Substr(prediction))
        sink(window_left[-1])
    else:
      if is_replaceable_punct(window_right[0].text):
          skip(window_right.popleft())

    skip_next = is_replaceable_punct(window_right[0].text)

    while len(window_left) != params['INPUT_WORDS_CNT_LEFT'] - 1:
      token = window_left.popleft()

    if log: print(d_as_str(window_left).rjust(100), "      ", d_as_str(window_right))

  for i in window_right:
    sink(i)
  sink_remaining()
  ress = "".join(res)
  return ress

In [22]:
import time
before = time.time()
await infer_optimal(params, "тест " * 100)
print(time.time() - before)

0.7259631156921387


In [26]:
import time
torch.set_num_threads(1)
before = time.time()
apply_te("тест " * 100)
print(time.time() - before)

0.09819602966308594
